In [1]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import torch

# Core Libraries
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

# Operating System Interaction
import os
import sys

# Machine Learning Frameworks
import torch
from torchvision import datasets
from torch.utils.data import Dataset, DataLoader


# Model Building and Initialization
import torch.nn as nn
from torch.nn.init import kaiming_normal_

# Data Loading and Dataset Handling
from torchvision.datasets import ImageFolder
from torch.utils.data import random_split, Subset
from PIL import Image
import json

# Cross-Validation and Metrics
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score, roc_curve, auc, accuracy_score, confusion_matrix
from scipy.special import expit as sigmoid

# Visualization and Display
from matplotlib.animation import FuncAnimation
from matplotlib.colors import Normalize
import matplotlib.patches as patches

from IPython.display import HTML
from astropy.visualization import ImageNormalize, SqrtStretch
import seaborn as sns
import sunpy.visualization.colormaps as cm

# Miscellaneous
import random
from tqdm import tqdm

Format of input file

In [2]:
import json

data = {
    "categories": [ #id 1 will be changed from airplane to jet 
        {"id": 1, "name": "airplane", "encode_name": "n02691156"},
        {"id": 2, "name": "antelope", "encode_name": "n02419796"},
        {"id": 3, "name": "bear", "encode_name": "n02131653"},
        {"id": 4, "name": "bicycle", "encode_name": "n02834778"},
        {"id": 5, "name": "bird", "encode_name": "n01503061"},
        {"id": 6, "name": "bus", "encode_name": "n02924116"},
        {"id": 7, "name": "car", "encode_name": "n02958343"},
        {"id": 8, "name": "cattle", "encode_name": "n02402425"},
        {"id": 9, "name": "dog", "encode_name": "n02084071"},
        {"id": 10, "name": "domestic_cat", "encode_name": "n02121808"},
        {"id": 11, "name": "elephant", "encode_name": "n02503517"},
        {"id": 12, "name": "fox", "encode_name": "n02118333"},
        {"id": 13, "name": "giant_panda", "encode_name": "n02510455"},
        {"id": 14, "name": "hamster", "encode_name": "n02342885"},
        {"id": 15, "name": "horse", "encode_name": "n02374451"},
        {"id": 16, "name": "lion", "encode_name": "n02129165"},
        {"id": 17, "name": "lizard", "encode_name": "n01674464"},
        {"id": 18, "name": "monkey", "encode_name": "n02484322"},
        {"id": 19, "name": "motorcycle", "encode_name": "n03790512"},
        {"id": 20, "name": "rabbit", "encode_name": "n02324045"},
        {"id": 21, "name": "red_panda", "encode_name": "n02509815"},
        {"id": 22, "name": "sheep", "encode_name": "n02411705"},
        {"id": 23, "name": "snake", "encode_name": "n01726692"},
        {"id": 24, "name": "squirrel", "encode_name": "n02355227"},
        {"id": 25, "name": "tiger", "encode_name": "n02129604"},
        {"id": 26, "name": "train", "encode_name": "n04468005"},
        {"id": 27, "name": "turtle", "encode_name": "n01662784"},
        {"id": 28, "name": "watercraft", "encode_name": "n04530566"},
        {"id": 29, "name": "whale", "encode_name": "n02062744"},
        {"id": 30, "name": "zebra", "encode_name": "n02391049"}
    ],

    "images":[
        {"file_name" : "DET/train/ILSVRC2014_train_0000/ILSVRC2014_train_00000663.JPEG", 
         "height": 166, 
         "width": 166, 
         "id": 1, #starting from 1
         "frame_id": -1,  #starting from 0 and reseting when changing video
         "video_id": 1, 
         "is_vid_train_frame": True}
    ],#all files (images)

    "annotations" :[
        {"id": 1, #one id per image even if it's in the same vid
         "video_id": 1, #goes up to the number of videos 
         "image_id": 1, 
         "category_id": 1, #will always be 1 as the jets will be classified as 1 
         "instance_id": 1, 
         "bbox": [1, 90, 496, 271], 
         "area": 134416, 
         "iscrowd": False, 
         "occluded": False, 
         "generated": False}
    ],

    "videos":[
        {"id": 1,
        "name": "VID/train/ILSVRC2015_VID_train_0000/ILSVRC2015_train_00000000", 
        "vid_train_frames": [9, 29, 49, 69, 89, 109, 129, 149, 169, 189, 209, 229, 249, 269, 289]} #take all frames maybe for training
    ]   
}

Take all labeled events and transform them into JET_train_eventx/imagex.JPG

In [3]:
import os

def list_files_in_folder(folder_path):
    """
    List all files in a folder in ascending order.
    
    Args:
    - folder_path: The path to the folder
    
    Returns:
    - file_names: A list of file names in the folder, sorted in ascending order
    """
    file_names = []
    for file_name in sorted(os.listdir(folder_path), key=lambda x: int(os.path.splitext(x)[0])):
        if os.path.isfile(os.path.join(folder_path, file_name)):
            file_names.append(file_name)
    return file_names

# Example usage:
folder_path = "../../../../jet_box_sophie/bbox_events_jets/000"
file_names = list_files_in_folder(folder_path)
print(file_names)
print(len(file_names))


['000_000.csv', '000_001.csv', '000_002.csv', '000_003.csv', '000_004.csv', '000_005.csv', '000_006.csv', '000_007.csv', '000_008.csv', '000_009.csv', '000_010.csv', '000_011.csv', '000_012.csv', '000_013.csv', '000_014.csv', '000_015.csv', '000_016.csv', '000_017.csv', '000_018.csv', '000_019.csv', '000_020.csv', '000_021.csv', '000_022.csv', '000_023.csv', '000_024.csv', '000_025.csv', '000_026.csv', '000_027.csv', '000_028.csv', '000_029.csv', '000_030.csv', '000_031.csv', '000_032.csv', '000_033.csv', '000_034.csv', '000_035.csv', '000_036.csv', '000_037.csv', '000_038.csv', '000_039.csv', '000_040.csv', '000_041.csv', '000_042.csv', '000_043.csv', '000_044.csv', '000_045.csv', '000_046.csv', '000_047.csv', '000_048.csv', '000_049.csv', '000_050.csv', '000_051.csv', '000_052.csv', '000_053.csv', '000_054.csv', '000_055.csv', '000_056.csv', '000_057.csv', '000_058.csv', '000_059.csv', '000_060.csv', '000_061.csv', '000_062.csv', '000_063.csv', '000_064.csv', '000_065.csv', '000_066.

In [4]:
def generate_image_data(folder_path,data):

    video_id = len(data["videos"]) + 1 #everytime we change folder video id is increased by one
    vid_info = {
                "id": video_id,
                "name":folder_path[-13:].replace("\\", "/"),
                "vid_train_frames":[i for i in range(len(os.listdir(folder_path)))]
            }

    data["videos"].append(vid_info)

    frame_id=0

    
    #print(folder_path)
    for i, filename in enumerate(os.listdir(folder_path)):
        #print(filename)
        #IMAGE INFOS
        if filename.endswith(".JPEG"):
            image_id = len(data["images"]) + 1
            annot_id = len(data["annotations"]) + 1
            image_path = os.path.join(folder_path, filename)

            if i ==0:
                with Image.open(image_path) as img:
                    width, height = img.size
            

            image_info = {
                "file_name": os.path.relpath(image_path[7:],".").replace("\\", "/"),
                "height": height,
                "width": width,
                "id": image_id,
                "frame_id": frame_id,
                "video_id": video_id,
                "is_vid_train_frame": True
            }
            data["images"].append(image_info)


        #BOUNDING BOX INFOS (ANNOTATIONS)
            bbox_im=pd.read_csv(f"./Data/bbox_events_jets/{filename[:3]}/{filename[:3]}_{filename[4:7]}.csv").values.tolist()
           
            for bbox in bbox_im: #If multiple bbox for one image
                bbox_area=torch.tensor(bbox).view(1,4)
                area=torchvision.ops.box_area(bbox_area).item()
                width = bbox[2] - bbox[0]
                height = bbox[3] - bbox[1]
                # Create a list containing [x_min, y_min, width, height]
                bbox_list = [bbox[0], bbox[1], width, height]
                #print(bbox_list)
                annotation_info = {
                    "id": annot_id,
                    "video_id": video_id,
                    "image_id": image_id,
                    "category_id": 1,
                    "instance_id": 1,
                    "bbox": bbox_list, #in [x1,y1,x2,y2]                              
                    "area": area, 
                    "iscrowd": False,
                    "occluded": False,
                    "generated": False
                }
                data["annotations"].append(annotation_info)
                annot_id+=1
            frame_id+=1
            
    return data
# Function to generate data for each folder in a directory
def generate_folder_data(root_folder, data):
    for folder_name in os.listdir(root_folder):#All folder in folder path (VID) TRAIN or VAL
        folder_path = os.path.join(root_folder, folder_name)
        if os.path.isdir(folder_path):
            data=generate_image_data(folder_path,data)



data = {
    "categories": [ #id 1 will be changed from airplane to jet 
        {"id": 1, "name": "jet", "encode_name": "n02691156"},
        {"id": 2, "name": "antelope", "encode_name": "n02419796"},
        {"id": 3, "name": "bear", "encode_name": "n02131653"},
        {"id": 4, "name": "bicycle", "encode_name": "n02834778"},
        {"id": 5, "name": "bird", "encode_name": "n01503061"},
        {"id": 6, "name": "bus", "encode_name": "n02924116"},
        {"id": 7, "name": "car", "encode_name": "n02958343"},
        {"id": 8, "name": "cattle", "encode_name": "n02402425"},
        {"id": 9, "name": "dog", "encode_name": "n02084071"},
        {"id": 10, "name": "domestic_cat", "encode_name": "n02121808"},
        {"id": 11, "name": "elephant", "encode_name": "n02503517"},
        {"id": 12, "name": "fox", "encode_name": "n02118333"},
        {"id": 13, "name": "giant_panda", "encode_name": "n02510455"},
        {"id": 14, "name": "hamster", "encode_name": "n02342885"},
        {"id": 15, "name": "horse", "encode_name": "n02374451"},
        {"id": 16, "name": "lion", "encode_name": "n02129165"},
        {"id": 17, "name": "lizard", "encode_name": "n01674464"},
        {"id": 18, "name": "monkey", "encode_name": "n02484322"},
        {"id": 19, "name": "motorcycle", "encode_name": "n03790512"},
        {"id": 20, "name": "rabbit", "encode_name": "n02324045"},
        {"id": 21, "name": "red_panda", "encode_name": "n02509815"},
        {"id": 22, "name": "sheep", "encode_name": "n02411705"},
        {"id": 23, "name": "snake", "encode_name": "n01726692"},
        {"id": 24, "name": "squirrel", "encode_name": "n02355227"},
        {"id": 25, "name": "tiger", "encode_name": "n02129604"},
        {"id": 26, "name": "train", "encode_name": "n04468005"},
        {"id": 27, "name": "turtle", "encode_name": "n01662784"},
        {"id": 28, "name": "watercraft", "encode_name": "n04530566"},
        {"id": 29, "name": "whale", "encode_name": "n02062744"},
        {"id": 30, "name": "zebra", "encode_name": "n02391049"}
    ],

    "images":[
    ],#all files (images)

    "annotations" :[
    ],

    "videos":[
    ]   
}

# Generating data for each folder in the big folder
big_folder_path = "./Data/VID/TRAIN"
generate_folder_data(big_folder_path, data)

def print_tensor_keys(data):
    for key, value in data.items():
        if isinstance(value, torch.Tensor):
            print("Key:", key)
            print("Value (Tensor):", value)

# Add this function call before serializing to JSON
print_tensor_keys(data)
# Saving the modified data to a JSON file
with open("./annotations/imagenet_vid_train_joint_30.json", "w") as json_file:
    json.dump(data, json_file)


In [5]:
print(len(data["images"]))
print(len(data["annotations"]))

28764
28764


# Evaluation dataset

In [7]:
def generate_image_data(folder_path,data):

    video_id = len(data["videos"]) + 1 #everytime we change folder video id is increased by one
    vid_info = {
                "id": video_id,
                "name":folder_path[-7:].replace("\\", "/"),
                "vid_train_frames":[]
            }

    data["videos"].append(vid_info)

    frame_id=0

    
    #print(folder_path)
    for i, filename in enumerate(os.listdir(folder_path)):
        #print(filename)
        #IMAGE INFOS
        if filename.endswith(".JPEG"):
            image_id = len(data["images"]) + 1
            annot_id = len(data["annotations"]) + 1
            image_path = os.path.join(folder_path, filename)

            if i ==0:
                with Image.open(image_path) as img:
                    width, height = img.size
            

            image_info = {
                "file_name": image_path[11:].replace("\\", "/"),
                "height": height,
                "width": width,
                "id": image_id,
                "frame_id": frame_id,
                "video_id": video_id,
                "is_vid_train_frame": False
            }
            data["images"].append(image_info)


        #BOUNDING BOX INFOS (ANNOTATIONS)
            bbox_im=pd.read_csv(f"./Data/bbox_events_jets/{filename[:3]}/{filename[:3]}_{filename[4:7]}.csv").values.tolist()
           
            for bbox in bbox_im: #If multiple bbox for one image
                bbox_area=torch.tensor(bbox).view(1,4)
                area=torchvision.ops.box_area(bbox_area).item()
                width = bbox[2] - bbox[0]
                height = bbox[3] - bbox[1]
                # Create a list containing [x_min, y_min, width, height]
                bbox_list = [bbox[0], bbox[1], width, height]
                annotation_info = {
                    "id": annot_id,
                    "video_id": video_id,
                    "image_id": image_id,
                    "category_id": 1,
                    "instance_id": 1,
                    "bbox": bbox_list, #in [x1,y1,w,h]                              
                    "area": area, 
                    "iscrowd": False,
                    "occluded": False,
                    "generated": False
                }
                data["annotations"].append(annotation_info)
                annot_id+=1
            frame_id+=1
            
    return data
# Function to generate data for each folder in a directory
def generate_folder_data(root_folder, data):
    for folder_name in os.listdir(root_folder):#All folder in folder path (VID) TRAIN or VAL
        folder_path = os.path.join(root_folder, folder_name)
        if os.path.isdir(folder_path):
            data=generate_image_data(folder_path,data)



data = {
    "categories": [ #id 1 will be changed from airplane to jet 
        {"id": 1, "name": "jet", "encode_name": "n02691156"},
        {"id": 2, "name": "antelope", "encode_name": "n02419796"},
        {"id": 3, "name": "bear", "encode_name": "n02131653"},
        {"id": 4, "name": "bicycle", "encode_name": "n02834778"},
        {"id": 5, "name": "bird", "encode_name": "n01503061"},
        {"id": 6, "name": "bus", "encode_name": "n02924116"},
        {"id": 7, "name": "car", "encode_name": "n02958343"},
        {"id": 8, "name": "cattle", "encode_name": "n02402425"},
        {"id": 9, "name": "dog", "encode_name": "n02084071"},
        {"id": 10, "name": "domestic_cat", "encode_name": "n02121808"},
        {"id": 11, "name": "elephant", "encode_name": "n02503517"},
        {"id": 12, "name": "fox", "encode_name": "n02118333"},
        {"id": 13, "name": "giant_panda", "encode_name": "n02510455"},
        {"id": 14, "name": "hamster", "encode_name": "n02342885"},
        {"id": 15, "name": "horse", "encode_name": "n02374451"},
        {"id": 16, "name": "lion", "encode_name": "n02129165"},
        {"id": 17, "name": "lizard", "encode_name": "n01674464"},
        {"id": 18, "name": "monkey", "encode_name": "n02484322"},
        {"id": 19, "name": "motorcycle", "encode_name": "n03790512"},
        {"id": 20, "name": "rabbit", "encode_name": "n02324045"},
        {"id": 21, "name": "red_panda", "encode_name": "n02509815"},
        {"id": 22, "name": "sheep", "encode_name": "n02411705"},
        {"id": 23, "name": "snake", "encode_name": "n01726692"},
        {"id": 24, "name": "squirrel", "encode_name": "n02355227"},
        {"id": 25, "name": "tiger", "encode_name": "n02129604"},
        {"id": 26, "name": "train", "encode_name": "n04468005"},
        {"id": 27, "name": "turtle", "encode_name": "n01662784"},
        {"id": 28, "name": "watercraft", "encode_name": "n04530566"},
        {"id": 29, "name": "whale", "encode_name": "n02062744"},
        {"id": 30, "name": "zebra", "encode_name": "n02391049"}
    ],

    "images":[
    ],#all files (images)

    "annotations" :[
    ],

    "videos":[
    ]   
}

# Generating data for each folder in the big folder
big_folder_path = "./Data/VID/VAL"
generate_folder_data(big_folder_path, data)

# Saving the modified data to a JSON file
with open("./annotations/imagenet_vid_val.json", "w") as json_file:
    json.dump(data, json_file)


In [8]:
print(len(data["images"]))
print(len(data["annotations"]))

3288
3288


Calculating mean and std of dataset

In [14]:
import numpy as np
from PIL import Image
import os

# Define a function to calculate mean and standard deviation
def calculate_mean_std(dataset_dir):
    mean = [0., 0., 0.]
    std = [0., 0., 0.]
    num_images = 0

    # Recursively traverse through each folder in the dataset directory
    for root, dirs, files in os.walk(dataset_dir):
        for filename in files:
            if filename.endswith(('.JPEG')):
                # Load the image
                img = Image.open(os.path.join(root, filename))
                img = np.array(img) / 255.  # Convert pixel values to range [0, 1]

                # Calculate mean and standard deviation
                mean += np.mean(img, axis=(0, 1))
                std += np.std(img, axis=(0, 1))

                num_images += 1
        print(f"Currently :{int(num_images/28764*100)} %")
    mean /= num_images
    std /= num_images

    return mean, std

# Example usage
dataset_dir = './Data/VID/TRAIN/'
mean, std = calculate_mean_std(dataset_dir)
print("Mean:", mean)
print("Standard Deviation:", std)


Currently :0 %
Currently :0 %
Currently :0 %
Currently :0 %
Currently :0 %
Currently :0 %
Currently :0 %
Currently :0 %
Currently :1 %
Currently :1 %
Currently :1 %
Currently :1 %
Currently :2 %
Currently :2 %
Currently :2 %
Currently :2 %
Currently :2 %
Currently :2 %
Currently :3 %
Currently :3 %
Currently :3 %
Currently :3 %
Currently :3 %
Currently :3 %
Currently :3 %
Currently :3 %
Currently :4 %
Currently :4 %
Currently :4 %
Currently :4 %
Currently :4 %
Currently :4 %
Currently :4 %
Currently :4 %
Currently :5 %
Currently :5 %
Currently :5 %
Currently :5 %
Currently :5 %
Currently :5 %
Currently :5 %
Currently :6 %
Currently :6 %
Currently :6 %
Currently :6 %
Currently :6 %
Currently :6 %
Currently :6 %
Currently :6 %
Currently :6 %
Currently :6 %
Currently :6 %
Currently :6 %
Currently :7 %
Currently :7 %
Currently :7 %
Currently :7 %
Currently :7 %
Currently :7 %
Currently :7 %
Currently :8 %
Currently :8 %
Currently :8 %
Currently :8 %
Currently :8 %
Currently :8 %
Currently 

Results  

Mean: [0.42501965 0.04450851 0.01185214]  
Standard Deviation: [0.21046326 0.10559631 0.05186318]